In [1]:
import openpathsampling as paths
import numpy as np

Load first frame from test pdb.

In [2]:
template = paths.tools.snapshot_from_pdb('../data/Alanine_solvated.pdb')

Create a complicated function.

In [3]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [4]:
cv0 = paths.CV_Function('func1', dist, center=center, np=np)
cv1 = paths.CV_Function('func1', dist, center=center, np=np, cv_wrap_numpy_array=True)
cv2 = paths.CV_Function('func1', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True)

Create storage to test save and load.

In [5]:
storage = paths.storage.Storage('can_be_deleted.nc', mode='w', template=template)

Save CV

In [6]:
storage.save([cv0, cv1, cv2])

Test function for reversed template.

In [7]:
print dist(template.reversed, center, np)

-1.08320000023


Get CV parameters for the function.

In [8]:
print cv0.return_parameters_from_template(template)
print cv1.return_parameters_from_template(template)
print cv2.return_parameters_from_template(template)

{'cv_return_simtk_unit': None, 'cv_return_type': 'numpy.float64', 'cv_return_shape': None}
{'cv_return_simtk_unit': None, 'cv_return_type': 'numpy.float64', 'cv_return_shape': ()}
{'cv_return_simtk_unit': None, 'cv_return_type': 'numpy.float64', 'cv_return_shape': ()}


Create another CV. This time using the `from_template` function.

In [9]:
print cv0([template, template])
print cv1([template, template])

[-1.0832000002264977, -1.0832000002264977]
[-1.0832 -1.0832]


In [10]:
print type(cv0([template, template]))
print type(cv0([template, template])[0])
print type(cv1([template, template]))
print type(cv1([template, template])[0])

<type 'list'>
<type 'numpy.float64'>
<type 'numpy.ndarray'>
<type 'numpy.float64'>


In [11]:
#! skip
print storage.cvs.variables['json'][0]
print storage.cvs.variables['json'][1]
print storage.cvs.variables['json'][2]

{"_cls": "CV_Function", "_dict": {"store_cache": true, "name": "func1", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTMtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB", "_module_vars": [], "_global_vars": []}, "scalarize_numpy_singletons": false, "wrap_numpy_array": false, "kwargs": {"np": {"_import": "numpy"}, "center": 1}, "requires_lists": false, "time_reversible": false}}
{"_cls": "CV_Function", "_dict": {"store_cache": true, "name": "func1", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTMtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB", "_module_vars": [], "_global_vars": []}, "scalarize_numpy_singleto

In [12]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]

In [13]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])

In [14]:
print storage.variables['snapshots_cv_0_values']
print storage.variables['snapshots_cv_1_values']
print storage.variables['snapshots_cv_2_values']

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_0_values(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (2,)
filling on, default _FillValue of 9.96920996839e+36 used

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_1_values(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (2,)
filling on, default _FillValue of 9.96920996839e+36 used

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_2_values(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (2,)
filling on, default _FillValue of 9.96920996839e+36 used



In [15]:
print storage.vars['snapshots_cv_0_values'][:]
print storage.vars['snapshots_cv_1_values'][:]
print storage.vars['snapshots_cv_2_values'][:]

[None, None]
[None, None]
[None, None]


Sync again.

In [16]:
cv0.sync()
cv1.sync()
cv2.sync()

Note that the third CV, which had `time_reversible=True` will automatically compute the values for the reversed snapshot, too. So after syncing we see two equal values.

In [17]:
print storage.vars['snapshots_cv_0_values'][:]
print storage.vars['snapshots_cv_1_values'][:]
print storage.vars['snapshots_cv_2_values'][:]

[-1.0832000002264977, None]
[-1.0832000002264977, None]
[-1.0832000002264977, -1.0832000002264977]


Since our function returns a numpy.float64 scalar we get these back (hopefully)

In [18]:
print type(storage.vars['snapshots_cv_0_values'][0])
print type(storage.vars['snapshots_cv_1_values'][0])
print type(storage.vars['snapshots_cv_2_values'][0])

<type 'numpy.float64'>
<type 'numpy.float64'>
<type 'numpy.float64'>


In [19]:
print cv0._store_dict.cache
print cv1._store_dict.cache
print cv2._store_dict.cache

LRUCache(1/0 of 1000000/0)
LRUCache(1/0 of 1000000/0)
LRUCache(2/0 of 1000000/0)
